In [17]:
import yaml
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm
import gc

In [2]:
def load_json_data():
    gc.collect()
    with open(f'../data/amazon_meta.json', 'r') as json_file :
        for line in json_file :
            data = json.loads(line)
            yield data['asin']

In [18]:
def load_rating():
    gc.collect()
    with open(f'../data/books.json', 'r') as json_file :
        for line in json_file :
            data = json.loads(line)
            yield data['reviewerID'], data['asin']

In [19]:
arr = []
i = 0
for data in tqdm(load_rating()):
    arr.append(data)

51311621it [09:16, 92209.78it/s] 


In [20]:
df = pd.DataFrame(arr, columns=['user','item'])

In [21]:
cnt_user = df.groupby('user').agg({'item' : 'count'})

In [22]:
cnt_book = df.groupby('item').agg({'user' : 'count'})

In [23]:
user_more_h = cnt_user[cnt_user['item'] > 500]
book_more_h = cnt_book[cnt_book['user'] > 500]

In [24]:
user_list = user_more_h.index
book_list = book_more_h.index

In [25]:
df_t = df.loc[df['user'].isin(user_list)]
df_t = df_t.loc[df['item'].isin(book_list)]

In [26]:
books = df_t['item'].unique()

In [27]:
len(books)

9966

In [31]:
with open('../data/publisher.json', 'a') as f:
    for i in tqdm(range(996, len(books))) :
        asin = books[i]
        if 'B' in asin :
            continue
        try :
            response = requests.get(f"https://www.bookfinder.com/search/?keywords={asin}&currency=USD&destination=kr&mode=basic&classic=off&lang=en&st=sh&ac=qr&submit=")
            body = BeautifulSoup(response.text).find('body')
            publisher = body.select_one('.describe-isbn').text.split(', ')
            author = body.select_one('#bd-isbn > div > div.attributes > div:nth-child(2) > p > strong > a > span').text
            f.write(f'{{"isbn" : "{asin}", "publisher" : "{publisher[0]}", "year" : "{publisher[1]}", "author" : "{author}"}}\n')
        except:
            f.write(f'{{"isbn" : "{asin}", "publisher" : "None", "year" : "None", "author" : "None"}}\n')

100%|██████████| 8970/8970 [04:07<00:00, 36.25it/s] 


In [29]:
with open('../data/publisher.json', 'r') as f:
    for line in f.readlines() :
        print(line)
        break

{"isbn" : "0001384198", "publisher" : "HarperCollins Publishers", "year" : "1984", "author" : "Piper Watty"}

